In [ ]:
# default_exp config

from nbdev.showdoc import show_doc
from utilities.ipynb_docgen import *


# Configuration data and basic functions
> Basic functions and configuration stuff

Implements:

- Cache
- Config
- MJD
- UTC, UTCnow

In [ ]:
#export
from astropy.time import Time
# from astropy.coordinates import SkyCoord, Angle
import astropy.units as u
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple
import os, sys
import numpy as np
import pickle

In [ ]:
#export
class Cache(dict):
    """
    Manage a file cache

    - `path` -- string or `filepath` object <br> This is the folder where the index and data files are saved.
    - `clear` -- set True to clear the cache on initialization

    This uses pickle to save objects, associated with a hashable key, which is used to index the
    filename in a file `index.pkl` in the same folder.

    The `__call__` function is a convenient way to use it, so one call may either store a new entry or retrieve an existing one.

    """

    def __init__(self, path, clear:bool=False):


        self.path = Path(path) if path else None
        if not self.path: return
        if not self.path.exists() :
            raise Exception(f'cache Path {self.path} does not exist ')

        self.index_file = self.path/'index.pkl'

        if self.path.exists():
            if clear:
                print('Clearing cache!')
                self.clear()
            else:
                self._load_index()

    def _dump_index(self):
        with open(self.index_file, 'wb') as file:
            pickle.dump(self, file)

    def _load_index(self):
        if not self.index_file.exists():
            self._dump_index()
            return
        with open(self.index_file, 'rb') as file:
            self.update(pickle.load(file))

    def add(self, key, object, exist_ok=False):
        if not self.path: return
        assert type(key)==str, f'Expect key to be a string, got {key}'
        if key  in self:
            if not exist_ok:
                print(f'Warning: cached object for key "{key}" exists', file=sys.stderr)
            filename = self[key]
        else:
            filename = self.path/f'cache_file_{hex(key.__hash__())[3:]}.pkl'
            self[key] = filename
            self._dump_index()

        with open(filename, 'wb') as file:
            pickle.dump(object, file )


    def get(self, key):
        if key not in self:
            return None
        filename = self[key]
        if not filename.exists():
            # perhaps deleted by another instance?
            print(f'File for Cache key {key} not found, removing entry', file='sys.stderr')
            selt.pop(key)
            return None
        with open(filename, 'rb') as file:
            ret = pickle.load(file)
        return ret

    def clear(self):
        if not self.path: return
        for f in self.path.iterdir():
            if f.is_file:
                f.unlink()
        super().clear()

        self._dump_index()

    def remove(self, key):
        """remove entry and associated file"""
        if not self.path: return
        if key not in self:
            print(f'Cache: key {key} not found', file=sys.stderr)
            return
        filename = self[key]
        try:
            filename.unlink()
        except:
            print(f'Failed to unlink file {filename}')
        super().pop(key)
        self._dump_index()


    def __call__(self, key, func, *pars, description='', overwrite=False, **kwargs,
                ):
        """
        One-line usage interface for cache use

        - `key` -- key to use, usually a string. Must be hashable <br>
            If None, ignore cache and return the function evaluation
        - `func` -- user function that will return an object that can be pickled
        - `pars`, `kwargs` -- pass to `func`
        - `description` -- optional string that will be printed
        - `overwrite` -- if set, overwrite previous entry if exists

        Example:
        <pre>
        mycache = Cache('/tmp/thecache', clear=True)

        def myfun(x):
            return x

        result = mycache('mykey', myfun, x=99,  description='My data')

        </pre>

        """

        if key is None:
            return func(*pars, **kwargs)

        ret = self.get(key)
        if description:
            print(f'{description}: {"Saving to" if key not in self or overwrite else "Restoring from"} cache', end='')
            print('' if key == description else f' with key "{key}"')

        if ret is None or overwrite:
            ret = func(*pars, **kwargs)
            self.add(key, ret, exist_ok=overwrite)
        return ret

    def show(self, starts_with=''):
        import datetime
        if not self.path: return 'Cache not enabled'
        title = 'Cache contents' if not starts_with else f'Cache entries starting with {starts_with}'
        s = f'{title}\n {"key":30}   {"size":>10}  {"time":20} {"name"}, in folder {self.path}\n'
        for name, value in self.items():
            if name is None or not name.startswith(starts_with) : continue
            try:
                stat = value.stat()
                size = stat.st_size
                mtime= str(datetime.datetime.fromtimestamp(stat.st_mtime))[:16]
                s += f'  {name:30s}  {size:10}  {mtime:20} {value.name}\n'
            except Exception as msg:
                s += f'{name} -- file not found'
        return s

    def __str__(self):
        return self.show()

In [ ]:
show_doc(Cache.__call__)

<h4 id="Cache.__call__" class="doc_header"><code>Cache.__call__</code><a href="__main__.py#L97" class="source_link" style="float:right">[source]</a></h4>

> <code>Cache.__call__</code>(**`key`**, **`func`**, **\*`pars`**, **`description`**=*`''`*, **`overwrite`**=*`False`*, **\*\*`kwargs`**)

One-line usage interface for cache use

- `key` -- key to use, usually a string. Must be hashable <br>
    If None, ignore cache and return the function evaluation
- `func` -- user function that will return an object that can be pickled
- `pars`, `kwargs` -- pass to `func`
- `description` -- optional string that will be printed
- `overwrite` -- if set, overwrite previous entry if exists

Example:
<pre>
mycache = Cache('/tmp/thecache', clear=True)

def myfun(x):
    return x

result = mycache('mykey', myfun, x=99,  description='My data')

</pre>

### Cache test

In [ ]:
#collapse_hide
def cache_test(path):
    c = Cache(path, clear=True)

    # simmple interface
    c.add('one', 'one');
    c.add('two', 'two')
    c.add('two', 'two') # getnerates warning
    if path is not None:
        assert c.get('two') == 'two'

    # test function interface
    func = lambda x:f'value: {x}'
    
    r1 = c('four',  func,  4, description='Test')
    r2 = c('four',  func,  5,  description='Test') #should not get called
    assert c.path is None or r1==r2, f'{r1}, {r2}'
    
    # remaving an entry
    print(f'Before remove:\n{c}')
    assert 'four' in c
    c.remove('four')
    assert 'four' not in c
    c.clear()

test_path = Path('/tmp/cache_test')
test_path.mkdir(exist_ok=True)
cache_test(test_path)


Clearing cache!
Test: Saving to cache with key "four"
Test: Restoring from cache with key "four"
Before remove:
Cache contents
 key                                    size  time                 name, in folder /tmp/cache_test
  one                                     18  2021-08-01 15:20     cache_file_d7a97129dbf65.pkl
  two                                     18  2021-08-01 15:20     cache_file_10d19c6307109a27.pkl
  four                                    23  2021-08-01 15:20     cache_file_41d038288807144b.pkl



In [ ]:
# export
class Config():
    defaults=\
    """
        verbose         : 1
            
        datapath        : None # where to find data--must be set
        cachepath       : None # cache location -- will be /tmp/wtlike_cache if not set

        # data cuts, processing
        radius          : 4
        cos_theta_max   : 0.4
        z_max           : 100
        offset_size     : 2.e-06  # scale factor used for event time

        # binning
        energy_edge_pars : [2,6,17] # pars for np.logspace
        etypes          : [0, 1] # front, back
        nside           : 1024  
        nest            : True

        # data selection for cell creation
        week_range      : []
        time_bins       : [0, 0, 7]
        exp_min         : 5

        # cell fitting
        use_kerr        : False
        likelihood_rep  : poisson
        poisson_tolerance : 0.2

    """
    
    def __init__(self, **kwargs):
        import yaml
        from yaml import SafeLoader
        
        # parameters: first defaults, then from ~/.config/wtlike/config.yaml, then kwars
        pars = yaml.load(self.defaults, Loader=SafeLoader)
        dp = Path('~/.config/wtlike/config.yaml').expanduser()
        if dp.is_file():
            userpars = yaml.load(open(dp,'r'), Loader=SafeLoader)
            pars.update(userpars)
            #print(f'update from user file {dp}: {userpars}')
        pars.update(kwargs)
        
        self.__dict__.update(pars)
        
        self.energy_edges = ee=np.logspace(*self.energy_edge_pars)
        self.energy_bins = np.sqrt(ee[1:] * ee[:-1])
        if not self.week_range:
            self.week_range = (None, None)
        
       # set up, check files paths
        self.error_msg=''
        if self.datapath is None:
            self.error_msg+='\ndatapath must be a folder with wtlike data'
        self.datapath = df = Path(self.datapath).expanduser()
        if not (self.datapath.is_dir() or  self.datapath.is_symlink()):
            self.error_msg+=f'\ndata_folder "{df}" is not a directory or symlink'
        subs = 'aeff_files weight_files data_files'.split()
        for sub in subs:
            if not ( (df/sub).is_dir() or  (df/sub).is_symlink()) :
                self.error_msg+=f'\n{df/sub} is not a directory or symlink'

        if self.cachepath is None:
            self.cachepath = Path('/tmp/wtlike_cache')
            os.makedirs(self.cachepath, exist_ok=True)
        else:
            self.cachepath =  Path(self.cachepath).expanduser()
        if not self.cachepath.is_dir():
            self.error_msg +=f'cachepath {self.cachepath} is not a folder.'
    
    @property
    def cache(self):
        if not hasattr(self, '_cache'):
            self._cache = Cache(self.cachepath, clear=False)
        return self._cache

    @property
    def valid(self):
        if len(self.error_msg)==0: return True
        print(f'wtlike configuration is invalid:\n{self.error_msg}',file=sys.stderr)
        return False

    def __str__(self):
        s = 'Configuration parameters \n'
        for name, value in self.__dict__.items():
            if name=='files' or name.startswith('_'): continue
            s += f'  {name:15s} : {value}\n'
        return s

    def __repr__(self): return str(self)
    def get(self, *pars): return self.__dict__.get(*pars)



In [ ]:
show_doc(Config)

<h2 id="Config" class="doc_header"><code>class</code> <code>Config</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>Config</code>(**\*\*`kwargs`**)



In [ ]:
#collapse-hide
def summary():
    """
    
    #### config.Config -- parameters are from three sources:
    - defaults
    - the file `~/.config/wtlike/config.yaml` if it exists
    - keyword args in Config constructor. For example, to suppress all printout:
        ```
    config = Config(verbose=0)
    ```
    
    ##### Config defaults
    This is yaml-format, corresponding to `config.yaml`.
    {config_defaults}
    
    
    #### Config contents as set up here
    {config_text}   


    
    ##### config.cache -- a file cache
    The class `Cache`, available from `config.cache` implements a file cache in the folder
    `{config.cachepath}`
    
    {cache_text}
    """
    config = Config()
    config_defaults = Config.defaults
    config_text = monospace(config, summary='config parameter list')

 
    cache_text = monospace(config.cache, 'current cache contents' )


    return locals()
nbdoc(summary) 


#### config.Config -- parameters are from three sources:
- defaults
- the file `~/.config/wtlike/config.yaml` if it exists
- keyword args in Config constructor. For example, to suppress all printout:
    ```
config = Config(verbose=0)
```

##### Config defaults
This is yaml-format, corresponding to `config.yaml`.

        verbose         : 1
            
        datapath        : None # where to find data--must be set
        cachepath       : None # cache location -- will be /tmp/wtlike_cache if not set

        # data cuts, processing
        radius          : 4
        cos_theta_max   : 0.4
        z_max           : 100
        offset_size     : 2.e-06  # scale factor used for event time

        # binning
        energy_edge_pars : [2,6,17] # pars for np.logspace
        etypes          : [0, 1] # front, back
        nside           : 1024  
        nest            : True

        # data selection for cell creation
        week_range      : []
        time_bins       : [0, 0, 7]
        exp_min         : 5

        # cell fitting
        use_kerr        : False
        likelihood_rep  : poisson
        poisson_tolerance : 0.2

    


#### Config contents as set up here
<details  class="nbdoc-description" >  <summary> config parameter list </summary>  <div style="margin-left: 5%"><pre>Configuration parameters <br>  verbose         : 1<br>  datapath        : /home/burnett/wtlike_data<br>  cachepath       : /home/burnett/wtlike_cache<br>  radius          : 4<br>  cos_theta_max   : 0.4<br>  z_max           : 100<br>  offset_size     : 2e-06<br>  energy_edge_pars : [2, 6, 17]<br>  etypes          : [0, 1]<br>  nside           : 1024<br>  nest            : True<br>  week_range      : (None, None)<br>  time_bins       : [0, 0, 7]<br>  exp_min         : 5<br>  use_kerr        : False<br>  likelihood_rep  : poisson<br>  poisson_tolerance : 0.2<br>  energy_edges    : [1.00000000e+02 1.77827941e+02 3.16227766e+02 5.62341325e+02<br> 1.00000000e+03 1.77827941e+03 3.16227766e+03 5.62341325e+03<br> 1.00000000e+04 1.77827941e+04 3.16227766e+04 5.62341325e+04<br> 1.00000000e+05 1.77827941e+05 3.16227766e+05 5.62341325e+05<br> 1.00000000e+06]<br>  energy_bins     : [1.33352143e+02 2.37137371e+02 4.21696503e+02 7.49894209e+02<br> 1.33352143e+03 2.37137371e+03 4.21696503e+03 7.49894209e+03<br> 1.33352143e+04 2.37137371e+04 4.21696503e+04 7.49894209e+04<br> 1.33352143e+05 2.37137371e+05 4.21696503e+05 7.49894209e+05]<br>  error_msg       : <br></pre></div> </details>   



##### config.cache -- a file cache
The class `Cache`, available from `config.cache` implements a file cache in the folder
`/home/burnett/wtlike_cache`

<details  class="nbdoc-description" >  <summary> current cache contents </summary>  <div style="margin-left: 5%"><pre>Cache contents<br> key                                    size  time                 name, in folder /home/burnett/wtlike_cache<br>  P88Y3278_data                    128097967  2021-07-30 07:11     cache_file_7cc64c0c03169077.pkl<br>  P88Y3243_data                     96003882  2021-07-31 06:10     cache_file_616b5b35718cbef9.pkl<br>  PSR J0633+1746_data              125596991  2021-07-29 15:49     cache_file_e5a52bc7bf78597.pkl<br>  P88Y4744_data                    155979422  2021-07-06 09:42     cache_file_313ff9a4a61b0dd.pkl<br>  PSR J1747-2958_data              153970472  2021-07-06 10:01     cache_file_67d7112d0fd9cf73.pkl<br>  Geminga_test                     134106948  2021-07-08 06:14     cache_file_110ee4a7bc8f2715.pkl<br>  P88Y3157_data                     94261671  2021-08-01 10:06     cache_file_4c35159676191122.pkl<br>  PSR J0534+2200_data              123229511  2021-07-09 10:41     cache_file_31da097aee4d5d1a.pkl<br>  Geminga_nokerr                   128071691  2021-07-29 14:33     cache_file_2905cf7a9572d1c4.pkl<br></pre></div> </details>


## Time conversion

- MET: mission elapsed time
- MJD: modified Julian date (days)
- UTC: ISO time
- UTCnow: current ISO time

In [ ]:
#export

day = 24*3600.
first_data=54683
#mission_start = Time('2001-01-01T00:00:00', scale='utc').mjd
# From a FT2 file header
# MJDREFI =               51910. / Integer part of MJD corresponding to SC clock S
# MJDREFF =  0.00074287037037037 / Fractional part of MJD corresponding to SC cloc
mission_start = 51910.00074287037

def MJD(arg):

    if type(arg)==str:
        while len(arg.split('-'))<3:
            arg+='-1'
        return Time(arg, format='iso').mjd

    "convert MET or UTC to MJD"
    return (mission_start + arg/day  )

def UTC(mjd):
    " convert MJD value to ISO date string"
    t=Time(mjd, format='mjd')
    t.format='iso'; t.out_subfmt='date_hm'
    return t.value

def UTCnow():
    from datetime import datetime
    t=datetime.utcnow()
    return f'UTC {t.year}-{t.month:02d}-{t.day} {t.hour:02d}:{t.minute:02d}'
       

In [ ]:
UTC(MJD(0)), UTC(first_data), UTCnow()

('2001-01-01 00:01', '2008-08-05 00:00', 'UTC 2021-08-1 21:33')

In [ ]:
assert UTC(MJD(0))=='2001-01-01 00:01'
assert MJD('2008')==54466

### Miscellaneous utilities

In [ ]:
#export
def bin_size_name(bins):
    """Provide a nice name, e.g., 'day' for a time interval
    """
    if np.isscalar(bins) :
        binsize = bins
    else:
        binsize = np.mean(bins)

    def check_unit(x):
        unit_table = dict(week=1/7, day=1, hour=24, min=24*60, s=24*3600)
        for name, unit in unit_table.items():
            t = x*unit
            r = np.mod(t+1e-9,1)
            if r<1e-6 or t>1:
                return t, name
        return x, 'day'
    n, unit =  check_unit(binsize)
    nt = f'{n:.0f}' if np.mod(n,1)<1e-2 else f'{n:.0f}'
    return f'{nt}-{unit}'# if n>1 else f'{unit}'

In [ ]:
# export
def decorate_with(other_func):
    def decorator(func):
        func.__doc__ += other_func.__doc__
        return func
    return decorator

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_data_man.ipynb.
Converted 02_effective_area.ipynb.
Converted 03_exposure.ipynb.
Converted 03_sources.ipynb.
Converted 04_load_data.ipynb.
Converted 04_simulation.ipynb.
Converted 05_source_data.ipynb.
Converted 06_poisson.ipynb.
Converted 07_loglike.ipynb.
Converted 08_cell_data.ipynb.
Converted 09_lightcurve.ipynb.
Converted 14_bayesian.ipynb.
Converted 90_main.ipynb.
Converted 99_tutorial.ipynb.
Converted index.ipynb.
Sun Aug  1 14:33:10 PDT 2021
